In [22]:
from fixerio import Fixerio # using the pre-release https://github.com/amatellanes/fixerio; that allows access key
from datetime import date, timedelta
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
# GETTING THE DATA
# at fixerio; use free plan
# we use the pre-release to be able to use access key
# put into pandas dataframe
# put into SQL database via sqlite3

# CLEANING THE DATA
# assumption not necessary, because highly 
# data comes from highly reputable source (ECB)
# data source widely used

# MAKE THE DATA USABLE
# put into data base
# write functions to query the database
# show and explain the data model
# show the conversion with the function

In [23]:
fxrio = Fixerio(access_key = '9befb24d8c3e96f66b2e3229c817af99')

In [25]:
# the task is to: load all exchanges rates from 01.01.2018 
# to now into a database
# we will make multiple historical queries to the api
# we have 1000 in the free plan and we need 501 queries
# also we have only one base currency available (EUR)
day1 = date(year=2018,month=1,day=1)
day2 = date(year=2019,month=5,day=16)

delta = day2 - day1 # timedelta

dates = []
for i in range(delta.days + 1):
    dates.append(day1 + timedelta(i))
    
print('We need to make {:.0f} queries'.format(len(dates)))


We need to make 501 queries


In [26]:
# let's query the API for every date once
#rates = []

#for d in dates:
#    rates.append(fxrio.historical_rates(d))


In [41]:
# put into dictionary
rates_dict = {}
for i,d in enumerate(dates):
    rates_dict[str(d)] = rates[i]

In [59]:
# now put in a data frame
# I suggest a table (pandas data frame) 
# with rows being dates
# variables are success, timestamp,
# and each currency is one variable
# let's get the names of the variables first
variables_1 = list(rates_dict['2018-01-20'].keys())[:-1]
variables_2 = list(rates_dict['2018-01-20']['rates'].keys())
variables = variables_1 + variables_2
print(variables)

# however building the data frame we saw that not always all currencies are available
# so we make sure we really query all possible currencies always, by forming the set
currencies = []
for d in dates:
    currencies = currencies + list(rates_dict[str(d)]['rates'].keys())
    
currencies = list(set(currencies))
currencies.sort()
print(currencies)
print(len(currencies))
print(len(variables_2))

# however: we have all possible currencies on Jan 20th

['success', 'timestamp', 'historical', 'base', 'date', 'AED', 'AFN', 'ALL', 'AMD', 'ANG', 'AOA', 'ARS', 'AUD', 'AWG', 'AZN', 'BAM', 'BBD', 'BDT', 'BGN', 'BHD', 'BIF', 'BMD', 'BND', 'BOB', 'BRL', 'BSD', 'BTC', 'BTN', 'BWP', 'BYN', 'BYR', 'BZD', 'CAD', 'CDF', 'CHF', 'CLF', 'CLP', 'CNY', 'COP', 'CRC', 'CUC', 'CUP', 'CVE', 'CZK', 'DJF', 'DKK', 'DOP', 'DZD', 'EGP', 'ERN', 'ETB', 'EUR', 'FJD', 'FKP', 'GBP', 'GEL', 'GGP', 'GHS', 'GIP', 'GMD', 'GNF', 'GTQ', 'GYD', 'HKD', 'HNL', 'HRK', 'HTG', 'HUF', 'IDR', 'ILS', 'IMP', 'INR', 'IQD', 'IRR', 'ISK', 'JEP', 'JMD', 'JOD', 'JPY', 'KES', 'KGS', 'KHR', 'KMF', 'KPW', 'KRW', 'KWD', 'KYD', 'KZT', 'LAK', 'LBP', 'LKR', 'LRD', 'LSL', 'LTL', 'LVL', 'LYD', 'MAD', 'MDL', 'MGA', 'MKD', 'MMK', 'MNT', 'MOP', 'MRO', 'MUR', 'MVR', 'MWK', 'MXN', 'MYR', 'MZN', 'NAD', 'NGN', 'NIO', 'NOK', 'NPR', 'NZD', 'OMR', 'PAB', 'PEN', 'PGK', 'PHP', 'PKR', 'PLN', 'PYG', 'QAR', 'RON', 'RSD', 'RUB', 'RWF', 'SAR', 'SBD', 'SCR', 'SDG', 'SEK', 'SGD', 'SHP', 'SLL', 'SOS', 'SRD', 'STD', 

In [62]:
# make a data frame and if a currency info is missing we will put NAN
rates_data = {}

for v in variables_1:
    x = []
    for d in dates:
        x.append(rates_dict[str(d)][v])
    rates_data[v] = x
    
for v in variables_2:
    x = []
    for i,d in enumerate(dates):
        try:
            x.append(rates_dict[str(d)]['rates'][v])
        except:
            print('Missing currency')
            x.append(np.nan)
            
    rates_data[v] = x
    
    
rates_data = pd.DataFrame(rates_data)
rates_data.to_csv('rates.csv')
rates_data.head(5)

Missing currency
Missing currency
Missing currency


,success,timestamp,historical,base,date,AED,AFN,ALL,AMD,ANG,...,XAU,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL
0,True,1514851199,True,EUR,2018-01-01,4.412975,83.395841,132.765995,581.043468,2.138499,...,0.000919,3.250891,0.843668,678.833208,119.261695,300.313919,14.886657,10814.903030,11.979648,387.308253
1,True,1514937599,True,EUR,2018-01-02,4.432021,83.854480,132.859093,583.566387,2.147927,...,0.000914,3.233631,0.844963,675.590291,119.366816,301.617897,15.003079,10861.858960,12.102800,388.989965
2,True,1515023999,True,EUR,2018-01-03,4.412236,83.501151,132.765008,581.041565,2.138437,...,0.000916,3.246223,0.842379,676.464081,119.263722,300.252857,14.846833,10814.902798,12.051223,387.306984
3,True,1515110399,True,EUR,2018-01-04,4.433811,84.028088,133.044473,583.850361,2.149437,...,0.000914,3.246777,0.845570,676.365741,119.294507,301.764670,14.838076,10867.144524,12.169518,389.179254
4,True,1515196799,True,EUR,2018-01-05,4.419187,83.731984,132.835347,581.876570,2.142214,...,0.000912,3.253030,0.843112,673.634003,119.660538,300.443705,14.784314,10830.416473,11.984495,387.863577


In [63]:
# alternatively we read the data from
# the csv if we have already queried the fixerio API and want to save our free API calls
rates_data = pd.read_csv('rates.csv')

Index(['BHD', 'BIF', 'BMD', 'BND', 'BOB', 'BRL', 'BSD', 'BTC', 'BTN', 'BWP'], dtype='object')

In [64]:
# example query for a price of 1 and EUR to USD conversion
rates_data['USD'][4]

1.203219

In [65]:
# now create database
connection = sqlite3.connect("conversion_rates.db")

In [66]:
# put it into the database
rates_data.to_sql(con=connection,name='conversion_rates',if_exists = 'replace')

/home/alexander/miniconda3/lib/python3.7/site-packages/pandas/core/generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [67]:
# query the database
connection = sqlite3.connect("conversion_rates.db")
 
def print_line_after(func):
    def wrapper(*args, **kwargs):
        res =  func(*args, **kwargs)
        print('Converted price is: {:.2f}'.format(res))
        print('\n')
        return(res)
    return wrapper

@print_line_after
def convert_price(from_currency,date_query,con,price=1,to_currency='USD'):
    '''
    INPUT:
    con: database to query
    date_query: string of date in format YYYY-MM-DD
    from_currency: currency in which price is given
    to_currency: currency to which price should be converted to
    
    VALUE:
    Converted price as float.
    It prints the SQL commands that were made.
    '''
    
    # getting cursor object to query
    cursor = connection.cursor()
    
    # check
    if from_currency == to_currency:
        raise ValueError('The from_currency and the to_currency cannot be the same.')
    
    # assign the query currencies
    if from_currency is not 'EUR' and to_currency is 'EUR':
        print('THIS CASE')
        to_curr_q = from_currency
        from_curr_q = to_currency
    else:
        to_curr_q = to_currency
        from_curr_q = from_currency
    
    # get the data:
    base_command = 'SELECT ' + to_curr_q + ' FROM conversion_rates '
    get_date_command = base_command + 'WHERE date = \'' + date_query + '\''
    print(get_date_command)
    cursor.execute(get_date_command) 
    result = cursor.fetchall() 
    
    # test that result is exactly of length 1
    assert(len(result) == 1)
    r = result[0]

    if from_currency == 'EUR':
        print
        return(float(r[0])*price)
    elif from_currency is not 'EUR' and to_currency is 'EUR':
        return(1/float(r[0])*price)
    elif from_currency is not 'EUR' and to_currency is not 'EUR':
        # get the second conversion
        base_command = 'SELECT ' + from_currency + ' FROM conversion_rates '
        get_date_command = base_command + 'WHERE date = \'' + date_query + '\''
        print(get_date_command)
        cursor.execute(get_date_command) 
        result2 = cursor.fetchall()
        assert(len(result2) == 1)
        r2 = result2[0]
        return((float(r[0])/float(r2[0])*price))
        


In [69]:
convert_price(from_currency='USD',date_query='2019-01-01',con=connection,price=1,to_currency='GBP')
convert_price(from_currency='EUR',date_query='2018-05-23',con=connection,price=1,to_currency='GBP')
convert_price(from_currency='BTC',date_query='2018-08-24',con=connection,price=1,to_currency='GBP')

SELECT GBP FROM conversion_rates WHERE date = '2019-01-01'
SELECT USD FROM conversion_rates WHERE date = '2019-01-01'
Converted price is: 0.78


SELECT GBP FROM conversion_rates WHERE date = '2018-05-23'
Converted price is: 0.88


SELECT GBP FROM conversion_rates WHERE date = '2018-08-24'
SELECT BTC FROM conversion_rates WHERE date = '2018-08-24'
Converted price is: 5204.61




5204.609195402299